<a href="https://colab.research.google.com/github/thor4/neuralnets/blob/master/projects/1-CNN/step_5_generate_figure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualize results

## 1: Load confidence & accuracy results
Run the cell to download a zip file from OSF then extract the following excel spreadsheets: `results_behavioral.xlsx`, `results_cifar10.xlsx`, `results_van3.xlsx`, and `results_vgg16-raw_conf.xlsx` into the current directory.

In [1]:
# @title Download results

import requests, os
from zipfile import ZipFile

print("Start downloading and unzipping `Behavioral results`...")
name = 'conf_acc_results'
fname = f"{name}.zip"
url = f"https://osf.io/xsf62/download" #osf share link
r = requests.get(url, allow_redirects=True)
with open(fname, 'wb') as fh:
  fh.write(r.content) #download file

with ZipFile(fname, 'r') as zfile:
  zfile.extractall() #extract contents

if os.path.exists(fname):
  os.remove(fname) #delete zip file
else:
  print(f"The file {fname} does not exist")

print("Download completed.")

Start downloading and unzipping `Behavioral results`...
Download completed.


These were the average confidence and accuracy results of the VGG16, CIFAR10 and Gabor models. The CIFAR10 and Gabor models used a vanilla CNN architecture. The VGG16 & CIFAR10 used transfer learning to fine-tune their top layers for the Gabor classification task. The Gabor model was trained specifically for the Gabor classifaction task. The behavioral results are also included and comprise the average confidence and accuracy results across 55 subjects.

Load necessary libraries.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

Load the spreadsheets into pandas dataframe.

In [3]:
df_beh = pd.read_excel("results_behavioral.xlsx") #mobilenetv2 logits
df_cif = pd.read_excel("results_cifar10.xlsx") #vanilla logits
df_van3 = pd.read_excel("results_van3.xlsx") #vanilla logits
df_vgg = pd.read_excel("results_vgg16-raw_conf.xlsx") #vanilla logits
print(df_beh.head())
print(df_cif.head())
print(df_van3.head())
print(df_vgg.head())

   Tilt  Contrast  Accuracy  Confidence
0   1.0         1  0.558909    2.058364
1   NaN         2  0.587818    2.251273
2   NaN         3  0.609091    2.465091
3   2.0         1  0.589455    2.080727
4   NaN         2  0.667091    2.295091
   Tilt  Contrast  Accuracy  Confidence
0   0.1      0.30     0.520    0.688855
1   NaN      0.45     0.511    0.683615
2   NaN      1.00     0.526    0.742383
3   0.2      0.30     0.497    0.696570
4   NaN      0.45     0.503    0.697424
   Tilt  Contrast  Accuracy  Confidence
0   0.1      0.30     0.650    1.112273
1   NaN      0.45     0.699    1.239503
2   NaN      1.00     0.835    1.958911
3   0.2      0.30     0.751    1.232105
4   NaN      0.45     0.858    1.678815
   Tilt  Contrast  Accuracy  Confidence
0   0.1      0.30     0.616    0.495122
1   NaN      0.45     0.574    0.544343
2   NaN      1.00     0.536    1.197594
3   0.2      0.30     0.598    0.483895
4   NaN      0.45     0.617    0.584853


In [ ]:
results.keys()

dict_keys(['__header__', '__version__', '__globals__', 'accuracy', 'confidence', 'subjects'])

In [ ]:
results['accuracy'].shape

(55, 3, 3)

## 2: Extract confidence and accuracy

In [ ]:
accuracy = results['accuracy']

Let's check the first subject's accuracy for all 9 tilt x contrast combinations.

In [ ]:
accuracy[0] 

array([[0.64, 0.69, 0.7 ],
       [0.65, 0.79, 0.84],
       [0.84, 0.95, 0.95]])

Now, let's average over all subjects to find the mean accuracy for each tilt x contrast condition.

In [ ]:
avg_acc = np.average(accuracy, axis=0)
avg_acc

array([[0.55890909, 0.58781818, 0.60909091],
       [0.58945455, 0.66709091, 0.70872727],
       [0.69890909, 0.78672727, 0.82636364]])

Do the same for confidence.

In [ ]:
avg_conf = np.average(results['confidence'], axis=0)
avg_conf

array([[2.05836364, 2.25127273, 2.46509091],
       [2.08072727, 2.29509091, 2.53545455],
       [2.17872727, 2.48781818, 2.792     ]])

In [ ]:
avg_acc.flatten() #unroll avg acc matrix

array([0.55890909, 0.58781818, 0.60909091, 0.58945455, 0.66709091,
       0.70872727, 0.69890909, 0.78672727, 0.82636364])

## 3: Create dataframe of results and save to Excel

Create a dataframe containing all average results with tilt x contrast assignments.

In [ ]:
df_results = pd.DataFrame({'Accuracy':avg_acc.flatten(),
                           'Confidence':avg_conf.flatten()})
df_results = df_results.assign(Tilt = [1, 1, 1, 2, 2, 2, 3, 3, 3],
                         Contrast = [1, 2, 3, 1, 2, 3, 1, 2, 3])

In [ ]:
df_results

,Accuracy,Confidence,Tilt,Contrast
0,0.558909,2.058364,1,1
1,0.587818,2.251273,1,2
2,0.609091,2.465091,1,3
3,0.589455,2.080727,2,1
4,0.667091,2.295091,2,2
5,0.708727,2.535455,2,3
6,0.698909,2.178727,3,1
7,0.786727,2.487818,3,2
8,0.826364,2.792000,3,3


Group the results by tilt and contrast for a clearer visualization.

In [ ]:
df_results_groupbyta = df_results.reset_index().set_index(['Tilt','Contrast'])
del df_results_groupbyta['index']
df_results_groupbyta

Accuracy  Confidence
Tilt Contrast                      
1    1         0.558909    2.058364
     2         0.587818    2.251273
     3         0.609091    2.465091
2    1         0.589455    2.080727
     2         0.667091    2.295091
     3         0.708727    2.535455
3    1         0.698909    2.178727
     2         0.786727    2.487818
     3         0.826364    2.792000

Save to disk.

In [ ]:
results_excel_filepath = os.path.join(os.getcwd(), 'results_behavioral.xlsx') #prep path to save to
df_results_groupbyta.to_excel(results_excel_filepath, index=True) #save to disk